In [ ]:
import warnings
warnings.filterwarnings('ignore')

import joblib
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from multiprocessing import Pool
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from tqdm import tqdm

# Carregando os datasets

Abaixo observa-se que ambos os datasets `ind_df` e `conn_df` possuem missing values. Em particular, chama a atenção o fato de que em `conn_df` os missing values concentram-se na variável target, a probabilidade de contaminação.

## conn_df

In [ ]:
conn_df = pd.read_csv('../data/raw/conexoes_espec.csv', sep=';')

In [ ]:
conn_df

In [ ]:
conn_df.info()

In [ ]:
conn_df.describe(include='all').T

In [ ]:
conn_df['grau'].unique()

In [ ]:
conn_df['proximidade'].unique()

## ind_df

In [ ]:
ind_df = pd.read_csv('../data/raw/individuos_espec.csv', sep=';')

In [ ]:
ind_df

In [ ]:
ind_df.info()

In [ ]:
ind_df.describe(include='all').T

In [ ]:
ind_df['estado_civil'].unique()

In [ ]:
ind_df['transporte_mais_utilizado'].unique()

## Combinando ind_df e conn_df

In [ ]:
df_raw = pd.merge(ind_df.rename(columns=lambda x: x + '_V2'),
                  conn_df, 
                  how='right',
                  left_on='name_V2', 
                  right_on='V2')
df_raw = pd.merge(ind_df.rename(columns=lambda x: x + '_V1'),
                  df_raw, 
                  how='right',
                  left_on='name_V1', 
                  right_on='V1')
df_raw = df_raw.drop(['V1', 'V2'], axis=1)
df_raw

# Utils

Modelos de classificação do scikit-learn não aceitam probabilidade como variável target, portanto a estratégia adotada para treinar modelos e estimar as probabilidades de contaminação foi:
1. aplicar a transformação logit (inversa da função sigmoide) às probabilidades, a fim de que possam assumir qualquer valor real;
2. treinar modelos de regressão utilizando os logits das probabilidades;
3. aplicar a transformação sigmoide aos resultados obtidos, e assim recuperar o target em termos de probabilidades; e
4. comparar a performance dos modelos usando a função de perda cross-entropy.

In [ ]:
def logit(p):
    return np.log(p / (1-p))

def sigmoid(y):
    ey = np.exp(y)
    return ey / (1+ey)

def cross_entropy_loss(y_true, y_pred):
    p, q = sigmoid(y_true), sigmoid(y_pred)
    return -(p*np.log(q) + (1-p)*np.log(1-q)).mean()

# Modelos

In [ ]:
categorical_cols = ['estado_civil_V1', 'estado_civil_V2',
                    'transporte_mais_utilizado_V1', 'transporte_mais_utilizado_V2',
                    'grau', 'proximidade']

boolean_cols = ['estuda_V1', 'trabalha_V1', 'pratica_esportes_V1',
                'estuda_V2', 'trabalha_V2', 'pratica_esportes_V2']

numerical_cols = ['idade_V1', 'qt_filhos_V1', 'IMC_V1',
                  'idade_V2', 'qt_filhos_V2', 'IMC_V2']

cols_to_drop = ['name_V1', 'name_V2', 'prob_V1_V2']

## Modelo Baseline

A fim de se obter rapidamente um modelo baseline cuja performance servirá como benchmark para os demais modelos, os missing values foram imputados da forma mais básica possível, de acordo com as seguintes estratégias:
1. variáveis categóricas $\leftarrow$ moda
2. variáveis booleanas $\leftarrow$ média
3. variáveis numéricas $\leftarrow$ mediana.

Para os demais modelos, foram testadas diferentes estratégias para imputar os missing values em `ind_df`. Já no caso de `conn_df`, foi utilizada sempre a mesma estratégia de imputar os missing values com as predições do modelo KNN Regressor (aplicado a uma versão do dataset original com a dimensionalidade reduzida pelo método PCA, buscando preservar cerca de 90% da variância explicada pelas features originais), treinado em um dataset com dados completos. Por fim, o target foi, em todos os casos, estimado utilizando-se a Regressão Linear a fim de que a comparabilidade das estratégias de imputação fosse preservada.


A única exceção a estas regras foi o último modelo, LightGBM Regressor, por se tratar de um que a presença de missing values entre os dados não representa um problema.

In [ ]:
df_bl = df_raw.copy(deep=True)

df_bl['prob_V1_V2'].fillna(df_bl['prob_V1_V2'].median(skipna=True), inplace=True)

In [ ]:
X_bl = df_bl.drop(cols_to_drop, axis=1)
y_bl = df_bl['prob_V1_V2']

X_bl_train_, X_bl_test_, y_bl_train, y_bl_test = train_test_split(X_bl,
                                                                  logit(y_bl),
                                                                  test_size=0.4,
                                                                  random_state=42)

In [ ]:
X_bl_train, X_bl_test = X_bl_train_.copy(deep=True), X_bl_test_.copy(deep=True)

mode_imp = SimpleImputer(strategy='most_frequent')
mean_imp = SimpleImputer(strategy='mean')
median_imp = SimpleImputer(strategy='median')

X_bl_train[categorical_cols] = mode_imp.fit_transform(X_bl_train_[categorical_cols])
X_bl_train[boolean_cols] = mean_imp.fit_transform(X_bl_train_[boolean_cols])
X_bl_train[numerical_cols] = median_imp.fit_transform(X_bl_train_[numerical_cols])

X_bl_test[categorical_cols] = mode_imp.transform(X_bl_test_[categorical_cols])
X_bl_test[boolean_cols] = mean_imp.transform(X_bl_test_[boolean_cols])
X_bl_test[numerical_cols] = median_imp.transform(X_bl_test_[numerical_cols])

In [ ]:
one_enc = OneHotEncoder(drop='first', sparse=False)
one_enc.fit(X_bl_train[categorical_cols])

one_hot_cols = one_enc.get_feature_names(categorical_cols)

X_bl_train[one_hot_cols] = one_enc.transform(X_bl_train[categorical_cols])
X_bl_train.drop(categorical_cols, axis=1, inplace=True)

X_bl_test[one_hot_cols] = one_enc.transform(X_bl_test[categorical_cols])
X_bl_test.drop(categorical_cols, axis=1, inplace=True)

In [ ]:
bl_model = LinearRegression().fit(X_bl_train, y_bl_train)

In [ ]:
train_loss = cross_entropy_loss(y_bl_train, bl_model.predict(X_bl_train))
test_loss = cross_entropy_loss(y_bl_test, bl_model.predict(X_bl_test))

print('baseline train loss:', train_loss)
print('baseline test loss:', test_loss)

## Modelo 1

A imputação dos missing values em `ind_df` foi feita utilizando o KNNImputer, um método computacionalmente bastante custoso. Em se tratando de um dataset com 1M de linhas, este processo teve uma duração média de 8 horas e 30 minutos. Por outro lado, a imputação do target em `conn_df` foi bem rápida, uma vez que a dimensionalidade do dataset foi reduzida de 31 features para 15 após aplicação do PCA.

In [ ]:
ind_df_1 = ind_df.copy(deep=True)

In [ ]:
ind_df_1

In [ ]:
cat_to_values = {'divorciado': 0,
                 'casado': 1,
                 'solteiro': 2,
                 'viuvo': 3,
                 'publico': 4,
                 'particular': 5,
                 'taxi': 6}

ind_df_1_ = ind_df_1.replace(cat_to_values)

scaler = MinMaxScaler()
ind_df_1_ = scaler.fit_transform(ind_df_1_.drop('name', axis=1))

In [ ]:
# %%time

# knn_imp = KNNImputer(weights='distance')

# ind_df_1_ = knn_imp.fit_transform(ind_df_1_)

In [ ]:
# ind_df_1_ = pd.DataFrame(scaler.inverse_transform(ind_df_1_), columns=ind_df_1_.columns)
# ind_df_1 = pd.concat([ind_df_1['name'], ind_df_1_], axis=1)

In [ ]:
# cols = ['estado_civil', 'transporte_mais_utilizado']
# values_to_cat = {v: k for (k, v) in cat_to_values.items()}
# ind_df_1[cols] = ind_df_1[cols].round().replace(values_to_cat)

In [ ]:
ind_df_1 = pd.read_csv('../data/processed/ind_df_filled_knn.csv')

ind_df_1

In [ ]:
conn_df

In [ ]:
df_1 = pd.merge(ind_df_1.rename(columns=lambda x: x + '_V2'),
                conn_df, 
                how='right',
                left_on='name_V2', 
                right_on='V2')
df_1 = pd.merge(ind_df_1.rename(columns=lambda x: x + '_V1'),
                df_1, 
                how='right',
                left_on='name_V1', 
                right_on='V1')
df_1 = df_1.drop(['V1', 'V2'], axis=1)
df_1

In [ ]:
scaler = MinMaxScaler()
pca = PCA(n_components=0.9, random_state=42)

df_1_ = pd.get_dummies(df_1.drop(cols_to_drop, axis=1), drop_first=True)
df_1_ = pca.fit_transform(scaler.fit_transform(df_1_))
pca_comps = [f'pc_{n+1}' for n in range(df_1_.shape[1])]
df_1_ = pd.DataFrame(scaler.fit_transform(df_1_), columns=pca_comps)

pd.Series(pca.explained_variance_ratio_, index=pca_comps).cumsum()

In [ ]:
df_1_ = df_1_.loc[:, :pca_comps[-1]]
df_1_['prob_V1_V2'] = df_1['prob_V1_V2']

df_1_

In [ ]:
df_1_.describe().T

In [ ]:
mask = df_1_['prob_V1_V2'].isna()
X_1_isna = df_1_.loc[mask].drop('prob_V1_V2', axis=1)
X_1_notna = df_1_.loc[~mask].drop('prob_V1_V2', axis=1)
y_1_notna = logit(df_1_.loc[~mask, 'prob_V1_V2'])

In [ ]:
knn_reg_imp = KNeighborsRegressor(weights='distance', n_jobs=-1)

knn_reg_imp.fit(X_1_notna, y_1_notna)

df_1_.loc[mask, 'prob_V1_V2'] = sigmoid(knn_reg_imp.predict(X_1_isna))

In [ ]:
df_1['prob_V1_V2'] = df_1_['prob_V1_V2']

X_1_, y_1 = df_1.drop(cols_to_drop, axis=1), df_1['prob_V1_V2']
X_1 = pd.get_dummies(X_1_, drop_first=True)

In [ ]:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1,
                                                            logit(y_1),
                                                            test_size=0.4,
                                                            random_state=42)

In [ ]:
lr_1_model = LinearRegression().fit(X_1_train, y_1_train)

In [ ]:
train_loss = cross_entropy_loss(y_1_train, lr_1_model.predict(X_1_train))
test_loss = cross_entropy_loss(y_1_test, lr_1_model.predict(X_1_test))

print('lr_1 train loss:', train_loss)
print('lr_1 test loss:', test_loss)

## Modelo 2

Observamos uma melhora significativa na performance do Modelo 1 em relação ao do Baseline, no entanto a um custo de mais de 8 horas de treinamento. Para a imputação dos missing values em `ind_df` no Modelo 2, tentamos então realizá-la utilizando EDA, e tentando encontrar padrões e correspondências entre os dados.

In [ ]:
ind_df_2 = ind_df.copy(deep=True)

In [ ]:
ind_df_2

In [ ]:
ind_df_2.isna().mean().sort_values()

In [ ]:
ind_df_2.corr().round(3)

Uma observação interessante acima é a correlação praticamente nula entre as variáveis `pratica_esportes`, `IMC` e as demais.

In [ ]:
ind_df_2['qt_filhos'].value_counts(normalize=True)

Por conta do baixo volume de observações com `qt_filhos > 1`, adotei como estratégia bucketizar todas essas observações. Assim, o número de categorias em `qt_filhos` caiu para 3, mas cada uma com um volume representativo de observações.

In [ ]:
ind_df_2.loc[ind_df_2['qt_filhos'] > 1, 'qt_filhos'] = 2
ind_df_2['qt_filhos'].value_counts(normalize=True)

In [ ]:
ind_df_2['estuda'].value_counts(normalize=True)

In [ ]:
ind_df_2['trabalha'].value_counts(normalize=True)

### EDA

#### `qt_filhos`, `estuda` e `trabalha`

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4), sharey=True)
sns.kdeplot(x='idade', hue='qt_filhos', data=ind_df_2, ax=ax[0])
sns.kdeplot(x='idade', hue='estuda', data=ind_df_2, ax=ax[1])
sns.kdeplot(x='idade', hue='trabalha', data=ind_df_2, ax=ax[2]);

Nos gráficos acima, vemos um padrão peculiar quando a idade é cerca de 16 anos, que pode ser explicado pelo fato de pessoas com 16 anos ou menos tendem a não ter filhos, estudarem e não trabalharem. Por conta disso, resolvi segmentar `idade` em um número suficiente de quantis que pudesse isolar este grupo de pessoas com menos de 16 anos. Como veremos abaixo, este segmento de pessoas, de fato, apresenta um comportamento diferente dos demais no que diz respeito às variáveis `qt_filhos`, `estuda` e `trabalha`. E, curiosamente, pessoas com 16 anos ou mais apresentam, em média, perfis de comportamento praticamente idênticos em relação a estas mesmas variáveis.

In [ ]:
ind_df_2['idade_quantis'] = pd.qcut(ind_df_2['idade'], q=15, labels=range(1, 16))

filhos_df = (ind_df_2
             .groupby('idade_quantis')['qt_filhos']
             .value_counts(normalize=True)
             .rename('pct')
             .reset_index())
estuda_df = (ind_df_2
             .groupby('idade_quantis')['estuda']
             .value_counts(normalize=True)
             .rename('pct')
             .reset_index())
trabalha_df = (ind_df_2
               .groupby('idade_quantis')['trabalha']
               .value_counts(normalize=True)
               .rename('pct')
               .reset_index())

fig, ax = plt.subplots(1, 3, figsize=(16, 4), sharey=True)
sns.barplot(x='idade_quantis', y='pct', hue='qt_filhos', data=filhos_df, ax=ax[0])
sns.barplot(x='idade_quantis', y='pct', hue='estuda', data=estuda_df, ax=ax[1])
ax[1].set_ylabel('')
sns.barplot(x='idade_quantis', y='pct', hue='trabalha', data=trabalha_df, ax=ax[2])
ax[2].set_ylabel('');

In [ ]:
ind_df_2['idade_cat'] = '16_mais'
ind_df_2.loc[ind_df_2['idade_quantis'] == 1, 'idade_cat'] = 'ate_16'
ind_df_2['idade_cat'] = pd.Categorical(ind_df_2['idade_cat'],
                                       categories=['ate_16', '16_mais'],
                                       ordered=True)

ind_df_2.drop('idade_quantis', axis=1, inplace=True)

ind_df_2.loc[ind_df_2['idade_cat'] == 'ate_16', 'idade'].describe().T

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4), sharey=True)

sns.barplot(x='idade_cat', y='idade', data=ind_df_2, hue='qt_filhos', ax=ax[0])
sns.barplot(x='idade_cat', y='idade', data=ind_df_2, hue='estuda', ax=ax[1])
ax[1].set_ylabel('')
sns.barplot(x='idade_cat', y='idade', data=ind_df_2, hue='trabalha', ax=ax[2])
ax[2].set_ylabel('');

In [ ]:
ind_df_2_ = ind_df_2.query('idade_cat == "16_mais"').copy(deep=True)

In [ ]:
ind_df_2_.corr().round(3)

Uma observação interessante acima é a correlação praticamente nula entre todas as variáveis, uma vez que consideramos apenas pessoas com 16 anos ou mais. E como veremos abaixo, o mesmo fenômeno ocorre em relação às variáveis categóricas `estado_civil` e `transporte_mais_utilizado`.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(9, 9), sharex=True, sharey=True)
fig.tight_layout()

cols_1 = ['qt_filhos', 'estuda', 'trabalha']
cols_2 = ['estado_civil', 'transporte_mais_utilizado']
for i, c1 in enumerate(cols_1):
    for j, c2 in enumerate(cols_2):
        tmp = (ind_df_2_
               .groupby(c2)[c1]
               .value_counts(normalize=True)
               .rename('pct')
               .mul(100)
               .reset_index())
        sns.barplot(x=c2, y='pct', hue=c1, data=tmp, ax=ax[i%3, j%2])
        if (c1 == 'qt_filhos') or (c1 == 'estuda'):
            ax[i%3, j%2].set_xlabel('')
        if c2 == 'estado_civil':
            ax[i%3, j%2].get_legend().remove()
        if c2 == 'transporte_mais_utilizado':
            ax[i%3, j%2].set_ylabel('')
            ax[i%3, j%2].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, title=c1)

Abaixo imputamos os missing values em `qt_filhos`, `estuda` e `trabalha` de acordo com as seguintes regras:
1. `idade < 16` $\leftarrow$ 0;
2. caso contrário $\leftarrow$ valor esperado das observações em que `idade > 15`;
3. no caso especial da variável `trabalha`, temos ainda uma informação adicional vinda da variável `grau` em `conn_df` que nos permite imputar com certeza missing values de observações em que `grau = trabalho`.

In [ ]:
cols = ['qt_filhos', 'estuda', 'trabalha']
for c in cols:
    pct_df = ind_df_2_[c].value_counts(normalize=True)
    mask_1, mask_2 = ind_df_2[c].isna(), (ind_df_2['idade_cat'] == 'ate_16')
    if c == 'qt_filhos':
        ind_df_2.loc[mask_1 & mask_2, c] = 0
    elif c == 'estuda':
        ind_df_2.loc[mask_1 & mask_2, c] = 1
    else:
        names = np.append(conn_df.loc[conn_df['grau'] == 'trabalho', 'V1'].unique(),
                          conn_df.loc[conn_df['grau'] == 'trabalho', 'V2'].unique())
        ind_df_2.loc[mask_1 & ind_df_2.index.isin(names), c] = 1
        ind_df_2.loc[mask_1 & mask_2, c] = 0
    ind_df_2[c].fillna(sum(pct_df.index*pct_df.values), inplace=True)

#### `transporte_mais_utilizado`

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(13, 11))
fig.tight_layout(h_pad=3, w_pad=4)

sns.kdeplot(x='idade', hue='transporte_mais_utilizado', data=ind_df_2, ax=ax[0, 0])

cols = ['idade_cat', 'estado_civil', 'qt_filhos', 'estuda', 'trabalha', 'pratica_esportes']
for n, c in enumerate(cols, start=1):
    tmp = (ind_df_2
           .groupby(c)['transporte_mais_utilizado']
           .value_counts(normalize=True)
           .rename('pct')
           .mul(100)
           .reset_index())
    g = sns.barplot(x=c, y='pct', hue='transporte_mais_utilizado', data=tmp, ax=ax[n//3, n%3])
    if c in ['qt_filhos', 'estuda', 'trabalha']:
        g.set_xticklabels(labels=['{:.1f}'.format(x) for x in sorted(tmp[c].unique())])
    ax[n//3, n%3].get_legend().remove()

sns.kdeplot(x='IMC', hue='transporte_mais_utilizado', data=ind_df_2, ax=ax[2, 1])

fig.delaxes(ax[2, 2]);

Como vemos acima, as demais variáveis não nos dão pistas de uma boa forma de imputar missing values em `transporte utilizado`. Portanto, dado o volume significativo de missing values nesta variável, equivalente à categoria `taxi`, eu optei por criar uma nova categoria apelidada de `outros` pra preencher estes valores.

In [ ]:
transp_pct_df = ind_df_2['transporte_mais_utilizado'].value_counts(normalize=True)
transp_pct_df

In [ ]:
ind_df_2['transporte_mais_utilizado'].fillna('outros', inplace=True)

#### `estado_civil`

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(13, 11))
fig.tight_layout(h_pad=3, w_pad=4)

tmp = (ind_df_2
       .groupby('idade_cat')['estado_civil']
       .value_counts(normalize=True)
       .rename('pct')
       .mul(100)
       .reset_index())

sns.kdeplot(x='idade', hue='estado_civil', data=ind_df_2, ax=ax[0, 0])
sns.barplot(x='idade_cat', y='pct', hue='estado_civil', data=tmp, ax=ax[0, 1])

cols = ['qt_filhos', 'estuda', 'trabalha', 'pratica_esportes', 'transporte_mais_utilizado']
for n, c in enumerate(cols, start=2):
    tmp = (ind_df_2_
           .groupby(c)['estado_civil']
           .value_counts(normalize=True)
           .rename('pct')
           .mul(100)
           .reset_index())
    g = sns.barplot(x=c, y='pct', hue='estado_civil', data=tmp, ax=ax[n//3, n%3])
    if c in ['qt_filhos', 'estuda', 'trabalha']:
        g.set_xticklabels(labels=['{:.1f}'.format(x) for x in sorted(tmp[c].unique())])
    ax[n//3, n%3].get_legend().remove()

sns.kdeplot(x='IMC', hue='estado_civil', data=ind_df_2_, ax=ax[2, 1])

fig.delaxes(ax[2, 2]);

Aqui, notamos fenômemo similar ao observado em `qt_filhos`, `estuda` e `trabalha`, no qual pessoas com menos de 16 anos são mais propensas a terem `estado_civil = solteiro`. E mais uma vez, considerando apenas observações com `idade` superior ou igual a 16 anos, observamos a ausência de correlação entre esta e as demais variáveis. Nesses casos, assim como já foi feito anteriormente, os missing values foram imputados com uma nova categoria apelidada de `outros`, dado o volume significativo destes.

In [ ]:
estcv_pct_df = ind_df_2_['estado_civil'].value_counts(normalize=True)
estcv_pct_df

In [ ]:
mask = ind_df_2['estado_civil'].isna()
ind_df_2.loc[mask & (ind_df_2['idade_cat'] == 'ate_16'), 'estado_civil'] = 'solteiro'
ind_df_2['estado_civil'].fillna('outros', inplace=True)

#### `pratica_esportes`

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(13, 11))
fig.tight_layout(h_pad=3, w_pad=4)

sns.kdeplot(x='idade', hue='pratica_esportes', data=ind_df_2, ax=ax[0, 0])

cols = ['idade_cat', 'estado_civil', 'qt_filhos', 'estuda', 'trabalha', 'transporte_mais_utilizado']
for n, c in enumerate(cols, start=1):
    tmp = (ind_df_2
           .groupby(c)['pratica_esportes']
           .value_counts(normalize=True)
           .rename('pct')
           .mul(100)
           .reset_index())
    g = sns.barplot(x=c, y='pct', hue='pratica_esportes', data=tmp, ax=ax[n//3, n%3])
    if c in ['qt_filhos', 'estuda', 'trabalha']:
        g.set_xticklabels(labels=['{:.1f}'.format(x) for x in sorted(tmp[c].unique())])
    ax[n//3, n%3].get_legend().remove()

sns.kdeplot(x='IMC', hue='pratica_esportes', data=ind_df_2, ax=ax[2, 1])

fig.delaxes(ax[2, 2]);

Caso similar ao de `tranporte_utilizado`, e com missing values imputados utilizando-se o valor esperado da variável.

In [ ]:
esportes_pct_df = ind_df_2_['pratica_esportes'].value_counts(normalize=True)
esportes_pct_df

In [ ]:
mask = ind_df_2['pratica_esportes'].isna()
ind_df_2['pratica_esportes'].fillna(sum(esportes_pct_df.index*esportes_pct_df.values), inplace=True)

#### `IMC` e `idade`

Para estas variáveis numéricas e contínuas, a estratégia de imputação utilizada foi pela mediana.

In [ ]:
# ind_df_2['IMC'].fillna(ind_df_2['IMC'].median(skipna=True), inplace=True)
# ind_df_2['idade'].fillna(ind_df_2['idade'].median(skipna=True), inplace=True)

# ind_df_2.drop('idade_ate16', axis=1, inplace=True)

In [ ]:
def fit_distribution(data, d):
    try:
        dist = getattr(stats, d)
    except:
        return 0
    param = dist.fit(data)
    ks_res = stats.kstest(data, d, args=param)
    return param, ks_res

In [ ]:
dist_discrete = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_discrete)]

with Pool() as p:
    res = p.starmap(fit_distribution, [(ind_df_2['idade'], dist_discrete)])

In [ ]:
results.sort(key=lambda x:float(x[2]), reverse=True)
for j in results[:10]:
    print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))

In [ ]:
dist_continuous = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_continuous)]
with Pool() as p:
    res = p.starmap(fit_distribution, [(ind_df_2['IMC'], dist_continuous)])

In [ ]:
results.sort(key=lambda x:float(x[2]), reverse=True)
for j in results[:10]:
    print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))

### Treinamento

Uma vez com `ind_df` completo, o combinamos com `conn_df` e procedemos com a redução da dimensionalidade com o método PCA, reduzindo o número de features, agora, de 31 para 18, seguido da imputação dos missing values no target com o KNN Regressor, como discutido anteriormente. Desta vez, no entanto, diferentemente do que ocorreu com o Modelo 1, o processo de imputação nesta segunda etapa levou cerca de 2 horas e 30 minutos. Ainda asssim, mesmo considerando o tempo gasto com toda a parte de EDA para completar `ind_df`, a estratégia de imputação utilizada no Modelo 2 teve uma duração total consideravelmente menor que aquela utilizada no Modelo 1, e como veremos abaixo, obtendo ainda uma taxa de performance apenas ligeiramente inferior.

In [ ]:
df_2 = pd.merge(ind_df_2.rename(columns=lambda x: x + '_V2'),
                conn_df, 
                how='right',
                left_on='name_V2', 
                right_on='V2')
df_2 = pd.merge(ind_df_2.rename(columns=lambda x: x + '_V1'),
                df_2, 
                how='right',
                left_on='name_V1', 
                right_on='V1')
df_2 = df_2.drop(['V1', 'V2'], axis=1)
df_2

In [ ]:
scaler = MinMaxScaler()
pca = PCA(n_components=0.9, random_state=42)

df_2_ = pd.get_dummies(df_2.drop(cols_to_drop, axis=1), drop_first=True)
df_2_ = pca.fit_transform(scaler.fit_transform(df_2_))
pca_comps = [f'pc_{n+1}' for n in range(df_2_.shape[1])]
df_2_ = pd.DataFrame(scaler.fit_transform(df_2_), columns=pca_comps)

pd.Series(pca.explained_variance_ratio_, index=pca_comps).cumsum()

In [ ]:
df_2_ = df_2_.loc[:, :pca_comps[-1]]
df_2_['prob_V1_V2'] = df_2['prob_V1_V2']

df_2_

In [ ]:
df_2_.describe().T

In [ ]:
mask = df_2_['prob_V1_V2'].isna()
X_2_isna = df_2_.loc[mask].drop('prob_V1_V2', axis=1)
X_2_notna = df_2_.loc[~mask].drop('prob_V1_V2', axis=1)
y_2_notna = logit(df_2_.loc[~mask, 'prob_V1_V2'])

In [ ]:
# %%time

# knn_reg_imp = KNeighborsRegressor(weights='distance', n_jobs=-1)

# knn_reg_imp.fit(X_2_notna, y_2_notna)

# df_2_.loc[mask, 'prob_V1_V2'] = sigmoid(knn_reg_imp.predict(X_2_isna))

In [ ]:
# df_2['prob_V1_V2'] = df_2_['prob_V1_V2']
df_2['prob_V1_V2'] = pd.read_csv('../data/processed/df_2_prob_V1_V2_filled_knn.csv')

X_2_, y_2 = df_2.drop(cols_to_drop, axis=1), df_2['prob_V1_V2']
X_2 = pd.get_dummies(X_2_, drop_first=True)

In [ ]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2,
                                                            logit(y_2),
                                                            test_size=0.4,
                                                            random_state=42)

lr_2_model = LinearRegression().fit(X_2_train, y_2_train)

In [ ]:
train_loss = cross_entropy_loss(y_2_train, lr_2_model.predict(X_2_train))
test_loss = cross_entropy_loss(y_2_test, lr_2_model.predict(X_2_test))

print('lr_2 train loss:', train_loss)
print('lr_2 test loss:', test_loss)

## LightGBM Regressor

Esta estratégia, dentre todas as utilizadas, é certamente a mais eficiente, pois não há necessidade de se preocupar em imputar os missing values. E mais do que isso, é um modelo que utiliza o fato de que existem missing values como informação para o treinamento e otimização da performance. Entretanto, trata-se de um modelo altamente complexo e de difícil interpretação.

In [ ]:
df_3 = pd.get_dummies(df_raw.drop(['name_V1', 'name_V2'], axis=1), drop_first=True)
X_3, y_3 = df_3.drop('prob_V1_V2', axis=1), df_3['prob_V1_V2']
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3,
                                                            logit(y_3),
                                                            test_size=0.4,
                                                            random_state=42)

In [ ]:
lgbm_reg = lgb.LGBMRegressor(random_state=42)
lgbm_reg.fit(X_3_train, y_3_train);

In [ ]:
train_loss = cross_entropy_loss(y_3_train, lgbm_reg.predict(X_3_train))
test_loss = cross_entropy_loss(y_3_test, lgbm_reg.predict(X_3_test))

print('lgbr train loss:', train_loss)
print('lgbr test loss:', test_loss)

# Estimando a taxa de contaminação para o resto da população

Feitas todas as considerações acima, optamos pela utilização do Modelo 2 por se tratar do melhor meio-termo quando ponderados os fatores: interpretabilidade, custo de modelagem e performance.

In [ ]:
df_half = df_2.copy(deep=True)

In [ ]:
df_half

In [ ]:
rev_cols = ['V2', 'V1', 'grau', 'proximidade']
rev_conn_df = conn_df[rev_cols].copy(deep=True).rename(columns={'V1': 'V2', 'V2': 'V1'})
rev_conn_df['prob_V1_V2'] = np.nan

In [ ]:
df_other_half = pd.merge(ind_df_2.rename(columns=lambda x: x + '_V2'),
                         rev_conn_df, 
                         how='right',
                         left_on='name_V2', 
                         right_on='V2')
df_other_half = pd.merge(ind_df_2.rename(columns=lambda x: x + '_V1'),
                         df_other_half, 
                         how='right',
                         left_on='name_V1', 
                         right_on='V1')
df_other_half = df_other_half.drop(['V1', 'V2'], axis=1)

df_other_half

In [ ]:
X_other_half_ = df_other_half.drop(cols_to_drop, axis=1)
X_other_half = pd.get_dummies(X_other_half_, drop_first=True)

In [ ]:
# lr_model = joblib.load('../data/processed/lr_2_model.pkl')
lr_model = lr_2_model

In [ ]:
df_other_half['prob_V1_V2'] = sigmoid(lr_model.predict(X_other_half))

In [ ]:
df_other_half

In [ ]:
df = df_half.append(df_other_half, ignore_index=True).drop(['name_V1', 'name_V2'], axis=1)

cols = ['idade_V1', 'qt_filhos_V1', 'estuda_V1', 'trabalha_V1', 'pratica_esportes_V1',
        'idade_V2', 'qt_filhos_V2', 'estuda_V2', 'trabalha_V2', 'pratica_esportes_V2']

df[cols] = df[cols].round()

df

## Importância das features

In [ ]:
scaler = StandardScaler()

xentropy_loss = make_scorer(cross_entropy_loss, greater_is_better=False)

feats_imp = permutation_importance(lr_model,
                                   scaler.fit_transform(X_2_train),
                                   y_2_train,
                                   scoring=xentropy_loss,
                                   n_jobs=-1,
                                   random_state=42)

In [ ]:
coefs_imp = (pd.
             DataFrame({'coefs': X_2_train.columns,
                        'avg_imp': feats_imp.importances_mean,
                        'std_imp': feats_imp.importances_std})
             .sort_values('avg_imp', ascending=False, ignore_index=True))

coefs_imp

## EDA

In [ ]:
prob_mean = df['prob_V1_V2'].mean()
prob_std = df['prob_V1_V2'].std()

In [ ]:
sns.histplot(x='prob_V1_V2', data=df)
plt.axvline(prob_mean - prob_std, color='black', linestyle=':')
plt.axvline(prob_mean + prob_std, color='black', linestyle=':');

In [ ]:
prob_mean = df['prob_V1_V2'].mean()
prob_std = df['prob_V1_V2'].std()

df['risco_contaminacao'] = 1
df.loc[df['prob_V1_V2'] < prob_mean - prob_std, 'risco_contaminacao'] = 0
df.loc[df['prob_V1_V2'] > prob_mean + prob_std, 'risco_contaminacao'] = 2

In [ ]:
df['risco_contaminacao'].value_counts(normalize=True)

In [ ]:
df.groupby('risco_contaminacao').apply(lambda x: x.describe().T)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

sns.kdeplot(x='idade_V1', data=df, hue='risco_contaminacao', ax=ax[0])
sns.kdeplot(x='idade_V2', data=df, hue='risco_contaminacao', ax=ax[1]);

In [ ]:
df['idade_V1_ate16'] = df['idade_V1'] < 16
# df['idade_V1_16mais'] = df['idade_V1'] >= 16

In [ ]:
# fig, ax = plt.subplots(16, 2, figsize=(10, 64))
fig, ax = plt.subplots(8, 2, figsize=(10, 32))
fig.tight_layout(h_pad=4, w_pad=4)

sns.kdeplot(x='idade_V1', data=df.query('idade_V1_ate16 == True'), hue='risco_contaminacao', ax=ax[0, 0])
# ax[0, 0].set_xlabel('')
ax[0, 0].set_ylabel('idade_V1_ate16')
sns.kdeplot(x='idade_V2', data=df.query('idade_V1_ate16 == True'), hue='risco_contaminacao', ax=ax[0, 1])
# ax[0, 1].set_xlabel('')
ax[0, 1].set_ylabel('')
# sns.kdeplot(x='idade_V1', data=df.query('idade_V1_16mais == True'), hue='risco_contaminacao', ax=ax[1, 0])
# ax[0, 1].set_ylabel('idade_V1_ate16')
# sns.kdeplot(x='idade_V2', data=df.query('idade_V1_ate16 == True'), hue='risco_contaminacao', ax=ax[0, 1])
# ax[0, 1].set_xlabel('')
# ax[0, 1].set_ylabel('')
# sns.kdeplot(x='idade_V2', data=df.query('idade_V1_16mais == True'), hue='risco_contaminacao', ax=ax[1, 1])
# ax[1, 1].set_ylabel('')

cols = ['estado_civil_V1', 'estado_civil_V2', 'qt_filhos_V1', 'qt_filhos_V2',
        'estuda_V1', 'estuda_V2', 'trabalha_V1', 'trabalha_V2', 'pratica_esportes_V1',
        'pratica_esportes_V2', 'transporte_mais_utilizado_V1', 'transporte_mais_utilizado_V2']

# for n, c in tqdm(enumerate(cols, start=2)):
for n, c in tqdm(enumerate(cols, start=2)):
    if n%2 == 0:
#         tmp = (df
#                .query('idade_V1_ate16 == True')
#                .groupby('risco_contaminacao')[c]
#                .value_counts(normalize=True)
#                .rename('pct')
#                .reset_index())
        
#         sns.barplot(x=c, y='idade_V1_ate16', data=df, hue='risco_contaminacao', ax=ax[2*(n//2), n%2])
#         ax[2*(n//2), n%2].get_legend().remove()
#         ax[2*(n//2), n%2].set_xlabel('')
        sns.barplot(x=c, y='idade_V1_ate16', data=df, hue='risco_contaminacao', ax=ax[n//2, n%2])
        ax[n//2, n%2].get_legend().remove()
#         ax[n//2, n%2].set_xlabel('')
        
#         sns.barplot(x=c, y='idade_V1_16mais', data=df, hue='risco_contaminacao', ax=ax[2*(n//2)+1, n%2])
#         ax[2*(n//2)+1, n%2].get_legend().remove()
        
    elif n%2 == 1:
#         tmp = (df
#                .query('idade_V1_ate16 == True')
#                .groupby('risco_contaminacao')[c]
#                .value_counts(normalize=True)
#                .rename('pct')
#                .reset_index())
        
#         sns.barplot(x=c, y='idade_V1_ate16', data=df, hue='risco_contaminacao', ax=ax[2*(n//2), n%2])
#         ax[2*(n//2), n%2].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, title='risco_contaminacao')
#         ax[2*(n//2), n%2].set_xlabel('')
#         ax[2*(n//2), n%2].set_ylabel('')
        sns.barplot(x=c, y='idade_V1_ate16', data=df, hue='risco_contaminacao', ax=ax[n//2, n%2])
        ax[n//2, n%2].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, title='risco_contaminacao')
#         ax[n//2, n%2].set_xlabel('')
#         ax[n//2, n%2].set_ylabel('')
        
#         sns.barplot(x=c, y='idade_V1_16mais', data=df, hue='risco_contaminacao', ax=ax[2*(n//2)+1, n%2])
#         ax[2*(n//2)+1, n%2].get_legend().remove()
#         ax[2*(n//2)+1, n%2].set_ylabel('')
        
sns.kdeplot(x='IMC_V1', data=df.query('idade_V1_ate16 == True'), hue='risco_contaminacao', ax=ax[7, 0])
# ax[7, 0].set_xlabel('')
ax[7, 0].set_ylabel('idade_V1_ate16')
# sns.kdeplot(x='IMC_V1', data=df.query('idade_V1_16mais == True'), hue='risco_contaminacao', ax=ax[15, 0])
# ax[15, 0].set_ylabel('idade_V1_ate16')
sns.kdeplot(x='IMC_V2', data=df.query('idade_V1_ate16 == True'), hue='risco_contaminacao', ax=ax[7, 1])
# ax[7, 1].set_xlabel('')
ax[7, 1].set_ylabel('');
# sns.kdeplot(x='IMC_V2', data=df.query('idade_V1_16mais == True'), hue='risco_contaminacao', ax=ax[15, 1])
# ax[15, 1].set_ylabel('');